<center style='font-size:28px;'><u><b>Pre-Processing</b></u></center>

1. [Project Description](#start)
    * [Data Description](#dd)
2. [EDA](#eda)
    * [Reading File](#refa)
    * [`days_employed`](#dysemp)
        * [Affinity Between `days_employed` and `total_income`](#affdystt)
    * [`education`](#education)
    * [`gender`](#gender)
    * [EDA Summation](#edasum)
3. [Pre-Processing](#pppp)
    * [`total_income` Missing Values](#toinmv)
    * [Hiding `None`s](#hidn)
    * [Casting](#cast)
    * [Duplicates](#dup)
    * [Categorisation](#catg)
        * [`purpose`](#purp)
        * [`children`](#child)
        * [`total_income`](#totti)
        * [Pre-Processing Summation](#ppppsum)
4. [Analysis](#analys)
    * [Is there a relation between having kids and repaying a loan on time?](#isarbhkaralot)
    * [Is there a relation between marital status and repaying a loan on time?](#msaralot)
    * [Is there a relation between income level and repaying a loan on time?](#bilaralot)
    * [How do different loan purposes affect on-time repayment of the loan?](#paotrotl)
5. [Conclusion](#conc)

# Project Description <a id="start"></a>

- Prepare a report for a bank's loan division, using existing bank's data
- Evaluate the ability of a potential borrower to repay their loan
- Specifically, check whether borrower's marital status or number of children has an impact on defaulting a loan

## Description of The Data <a id="dd"></a>

- `children` - the number of children in the family
- `days_employed` - how long the customer has been working
- `dob_years`- the customer’s age
- `education` - the customer’s education level
- `education_id` - identifier for the customer’s education
- `family_status` - the customer’s marital status
- `family_status_id` - identifier for the customer’s marital status
- `gender` - the customer’s gender
- `income_type` - the customer’s income type
- `debt` - whether the customer has ever defaulted on a loan
- `total_income` - monthly income
- `purpose` - reason for taking out a loan

In [1]:
import pandas as pd

# EDA <a id="eda"></a>

## Reading File <a id="refa"></a>

In [48]:
try:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/credit_scoring_eng.csv')
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Only 2 columns have missing values - `days_employed` and `total_income`. They both have the same number of them, possibly for the same records.

In [50]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


It's weird `days_employed` has non-int values. It's possible the loner total hours are divided by a standard 8 hour working day, so fractures are created. But thats a vague theory..

## `days_employed` <a id="dysemp"></a>

Min/max and mean/median values for `days_employed` are to be extracted -

In [41]:
max_days_employed = df.loc[df['days_employed'].idxmax()]['days_employed']
min_days_employed = df.loc[df['days_employed'].idxmin()]['days_employed']
span_days_employed = min_days_employed,max_days_employed                           
span_days_employed

(-18388.949900568383, 401755.40047533)

There's a big span of values. Some are negative.

In [42]:
negative_employ_ratio = df.loc[df['days_employed'] < 0]['days_employed'].count()/len(df)
f'{negative_employ_ratio:.1%} of days_employed values are negative'

'73.9% of days_employed values are negative'

The data seems weird. To account for falsely introducing the minus sign, absolute value of the column is calculated and the span is recalculated.

In [43]:
df['days_employed'] = df['days_employed'].abs()
max_days_employed_abs = df.loc[df['days_employed'].idxmax()]['days_employed']
min_days_employed_abs = df.loc[df['days_employed'].idxmin()]['days_employed']
span_days_employed_updated = min_days_employed_abs,max_days_employed_abs
f'updated extreme days_employed values: {span_days_employed_updated}'

'updated extreme days_employed values: (24.14163324048118, 401755.40047533)'

In [44]:
days_employed_abs_ratio = max_days_employed_abs / min_days_employed_abs
f'ratio of days_employed extreme values: {days_employed_abs_ratio}'

'ratio of days_employed extreme values: 16641.59986498587'

The span and ratio are so big, so this column is probably of better use when ignored: Its not a problem of measuring seconds/hours/years instead of days, as the latter ratio suggests.

Besides, it seems a lot of the values contain a lot of digits after the decimal dot, and thats another reason to doubt the data. In the context of `days_employed`, no property connected with so many digits after the decimal dot is in sight.

In [45]:
mean_days, median_days, std_days = df['days_employed'].describe()[['mean','50%','std']]

In [46]:
display(mean_days, median_days, std_days)

66914.72890682195

2194.220566878695

139030.8805274945

The mean is much greater than the median, which suggests a relatively small amount of values pushes the mean upwards.
If theres a lot of them close to the max value, and the next big clusters will be around the median, it will make for a reason to delete this cluster of big values next to the max value and keep the column after all.

In [47]:
print(df.loc[df['days_employed'] > 5*mean_days]['days_employed'].count()/len(df))
print(df.loc[df['days_employed'] < 5*mean_days]['days_employed'].count()/len(df))
print(df['days_employed'].isna().sum()/len(df))

0.1473170731707317
0.7516840882694541
0.10099883855981417


15% of the values are greater than 5 times the mean. The max_value is 6 * `mean_days` so no small cluster of points can be deleted in order to save the column.

### Affinity Between `days_employed` and `total_income` <a id="affdystt"></a>

Before deleting `days_employed`, a connection between its missing values to those of `total_income` is to be discovered. After all, they have the same number of missing values:

In [12]:
print(df.loc[df['total_income'].isna()]['days_employed'].isna().sum())

2174


There's an exact correlation between having missing `total_income` value to having missing `days_employed` value (21525 - 19351 = 2174). Presumably, the missing values at those rows originate from the loan seeker lack of job. Maybe theyr'e kids that
hadn't start working yet, but somehow have a bank account and they want a loan for a summer trip. Or maybe its people who are currently unemployed, so they write their previous `income_type` but leave the rest blank. As for businesses and civil servants - its strange their `total_income` is blank. They might be ashamed about those details and prefer not to disclose them if not necessary

In conclusion, `days employed` column is to be dropped.

In [13]:
df.drop('days_employed', axis=1, inplace=True)

## `education` <a id="education"></a>

See first the variety of education's subcategories:

In [14]:
print(df['education'].unique())

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


Conforming to lowercase and changing 'some college' to 'college' -

In [15]:
df['education'] = df['education'].str.lower()
df['education'] = df['education'].replace('some college', 'college')
df.education.value_counts()

secondary education    15233
bachelor's degree       5260
college                  744
primary education        282
graduate degree            6
Name: education, dtype: int64

## `gender` <a id="gender"></a>

In [16]:
print(df.gender.value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Theres one weird instance of a third gender -

In [17]:
display(df.loc[df['gender'] == 'XNA'])

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,24,college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


Besides `gender`, the row seems legit. But being a tiny fraction of the data, this row is better ignored.

In [18]:
df.drop(df.index[10701], inplace=True)

## EDA Summation <a id="edasum"></a>

- `days_employed` column was dropped due to unreasonable data
- In addition, 1 row was dropped due to suspicious data (genter of type XNA)

# Pre-Processing <a id="pppp"></a>

## `total_income` Missing Values <a id="toinmv"></a>

They came together with missing `days_employed` values so the leading theory is that its mostly people who are currently unemployed. Therefore, at present, they don't have any income at all. However, they each have affinity to their earlier profession - next job they'll have will probably be in the same field.
So there's some sense in filling missing values with the medians for the appropriate `income_type`.

In [19]:
df['total_income'] = df.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))

In [20]:
print(df.loc[df['total_income'].isna()]['income_type'].count())

0


## Hiding `None`s <a id="hidn"></a>

Data might contain hiding 'None's:

In [21]:
columns = df.columns
lst = []
for column in columns:
    tmp = df.loc[df[column] == 'None',column].count()
    lst.append(tmp)
lst

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

No 'None' hiding in the data

## Casting <a id="cast"></a>

There is no need for a float precision in `total_income`.

In [22]:
df.loc[:,'total_income'] = df.loc[:,'total_income'].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21524 non-null  int64 
 1   dob_years         21524 non-null  int64 
 2   education         21524 non-null  object
 3   education_id      21524 non-null  int64 
 4   family_status     21524 non-null  object
 5   family_status_id  21524 non-null  int64 
 6   gender            21524 non-null  object
 7   income_type       21524 non-null  object
 8   debt              21524 non-null  int64 
 9   total_income      21524 non-null  int64 
 10  purpose           21524 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


## Duplicates <a id="dup"></a>

In [23]:
df.duplicated().sum()

71

71 duplicates

In [24]:
df = df.drop_duplicates().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21453 non-null  int64 
 1   dob_years         21453 non-null  int64 
 2   education         21453 non-null  object
 3   education_id      21453 non-null  int64 
 4   family_status     21453 non-null  object
 5   family_status_id  21453 non-null  int64 
 6   gender            21453 non-null  object
 7   income_type       21453 non-null  object
 8   debt              21453 non-null  int64 
 9   total_income      21453 non-null  int64 
 10  purpose           21453 non-null  object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


21524 - 21453 = 71.

The duplicated data might have been present due to users applying twice.
71 duplicates in our data is one duplicate for every 300 applies for loan</center></font>

## Categorisation <a id="catg"></a>

### `purpose` <a id="purp"></a>

In [25]:
def purpose_group(purpose):
    if 'real estate' in purpose or 'house' in purpose or 'housing' in purpose or 'property' in purpose:
        return 'real estate'
    if 'education' in purpose or 'university' in purpose or 'educated' in purpose:
        return 'education'
    if 'wedding' in purpose:
        return 'wedding'
    if 'car' in purpose:
        return 'car'
    else:
        return 'unknown'
    
df['purpose_type'] = df['purpose'].apply(purpose_group)
df['purpose_type'].value_counts()

#A lot of 'unknown's. And their respective original purpose:

print(df.loc[df['purpose_type'] == 'unknown']['purpose'].value_counts())

#by tweeking the purpose_group function and rerunning the line above we categorise all the purposes until
#nothing gets printed.

Series([], Name: purpose, dtype: int64)


### `children` <a id="child"></a>

In [26]:
print(df.children.value_counts())

 0     14090
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


2 strange values are present: 20 and -1.
-1 might mean 1, Therefore it should be replaced by 1.

Right after typing a number using the keyboard's right side, one might type the key below it. If 1 or 3 are typed, something strange (right arrow or .) might generate an error when submitting a loan request.
But if 2 is typed, 20 will end up the number's sent, and the loan request form should be sent with no problem (20 could be valid).
So 20 to be replaced by 2.

In [27]:
df['children'] = df['children'].replace({-1: 1, 20: 2})
print(df.children.value_counts())

0    14090
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


### `total_income` <a id="totti"></a>

In [28]:
max_income = df.loc[df['total_income'].idxmax()]['total_income']
min_income = df.loc[df['total_income'].idxmin()]['total_income']
mean_income = df['total_income'].mean()
median_income  = df.total_income.median()
print(max_income,min_income,mean_income,median_income)

362496 3306 26450.314920990073 22815.0


From those values `total_income` is split into 4 categories: 2 for the half that earns below the median,
and another 2 categories for the other half who earns more than the median:

In [29]:
def income_categorizer(income):
    if income < 11400:
        return 1
    if income < 22800:
        return 2
    if income < 200000:
        return 3
    else:
        return 4
df['total_income_id'] = df['total_income'].apply(income_categorizer)

### Pre-Processing Summation <a id="ppppsum"></a>

- `total_income` missing values filled according to the borrower's `income_type`
- Problematic values in the children column are now fixed (0.05% of total data)
- `total_income` is categorised into 4 categories - 2 for under the median and 2 for above the median
- `purpose` various values reduced down to 4, in the `purpose_type` column

# Analysis <a id="analys"></a>

## Is there a relation between having kids and repaying a loan on time? <a id="isarbhkaralot"></a>

In [30]:
debt_by_child = df.groupby('children').agg({'debt': ['sum', 'count']})
debt_by_child['ratio'] = debt_by_child['debt']['sum'] / debt_by_child['debt']['count']
debt_by_child.sort_values(by='ratio', ascending=False, inplace=True)
print(debt_by_child)

          debt            ratio
           sum  count          
children                       
4            4     41  0.097561
2          202   2128  0.094925
1          445   4855  0.091658
3           27    330  0.081818
0         1063  14090  0.075444
5            0      9  0.000000


People with 5 children rarely take a loan, and when they do - they don't default it.
Maybe at their status some of their kids are old enough to help with the home chores more than "normal" kids,
so they can work more.
People with no children or 3 children have around 8% debt rate, and the others have roughly 20% chance more to get in debt.
People with 4 children rarely take loans as well, tough in their case - 10% of borrowers default the loan.

## Is there a relation between marital status and repaying a loan on time? <a id="msaralot"></a>

In [31]:
debt_by_family = df.groupby('family_status').agg({'debt': ['sum', 'count']})
debt_by_family['ratio'] = debt_by_family['debt']['sum'] / debt_by_family['debt']['count']
debt_by_family.sort_values(by='ratio', ascending=False, inplace=True)
print(debt_by_family)

                  debt            ratio
                   sum  count          
family_status                          
unmarried          274   2810  0.097509
civil partnership  388   4150  0.093494
married            931  12339  0.075452
divorced            85   1195  0.071130
widow / widower     63    959  0.065693


Widowers are the least likely to get into debt, with 6.5% default rate. Unmarried and civil partnership have 150% greater chance to default.
It might suggest age brings stability and low default rate

## Is there a relation between income level and repaying a loan on time? <a id="bilaralot"></a>

In [32]:
debt_by_income_id = df.groupby('total_income_id').agg({'debt': ['sum', 'count']})
debt_by_income_id['ratio'] = debt_by_income_id['debt']['sum'] / debt_by_income_id['debt']['count']
debt_by_income_id.sort_values(by='ratio', ascending=False, inplace=True)
print(debt_by_income_id)

                debt            ratio
                 sum  count          
total_income_id                      
4                  1     11  0.090909
2                701   8187  0.085624
3                930  11681  0.079616
1                109   1574  0.069250


Its pretty surprising that the people who earn the most actually have the highest debt rate. They might take much bigger lones than others, and therefore raise the chance of getting into debt. The lowest earning sector has the lowest debt rate, at 6.9%, 77% of highest earning sector default rate.

## How do different loan purposes affect on-time repayment of the loan? <a id="paotrotl"></a>

In [33]:
debt_by_purpose = df.groupby('purpose_type').agg({'debt':['sum', 'count']})
debt_by_purpose['ratio'] = debt_by_purpose['debt']['sum'] / debt_by_purpose['debt']['count']
debt_by_purpose.sort_values(by='ratio', ascending=False, inplace=True)
print(debt_by_purpose)

             debt            ratio
              sum  count          
purpose_type                      
car           403   4306  0.093590
education     370   4013  0.092200
wedding       186   2324  0.080034
real estate   782  10810  0.072340


Real-estate with the best debt rate at 7.2%, followed by wedding (8%), education (9.2%) and car (9.3%).

# Conclusion <a id="conc"></a>

The ideal borrower is a widow/er, with 5 kids or more, or 0 kids, that has a wage in the lowest quarter, and needs the lone for real estate related issues. He has around 2/3 chance to get in debt compared to the average loner